In [1]:
import os
import time
import datetime
import re
import shutil
from collections import deque
import argparse

import numpy as np
import torch
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from simulation.simulations.data_generator import DataGenerator
from TransformerMOT.util.misc import save_checkpoint, update_logs
from TransformerMOT.util.load_config_files import load_yaml_into_dotdict
from TransformerMOT.util.plotting import output_truth_plot, compute_avg_certainty, get_constrastive_ax, get_false_ax, \
    get_total_loss_ax, get_state_uncertainties_ax
from TransformerMOT.util.logger import Logger
from TransformerMOT.models.BOMTv1 import BOMT
from simulation.simulations.data_generator import DataGenerator, get_single_training_example


In [2]:
task_params = r"C:\Users\chiny\OneDrive - Nanyang Technological University\Y3S2 (Internship)\MultiTracking\configs\tasks\task2.yaml"
model_params = r"C:\Users\chiny\OneDrive - Nanyang Technological University\Y3S2 (Internship)\MultiTracking\configs\models\BOMTv1.yaml"

params = load_yaml_into_dotdict(task_params)
params.update(load_yaml_into_dotdict(model_params))

if params.general.pytorch_and_numpy_seed is None:
    random_data = os.urandom(4)
    params.general.pytorch_and_numpy_seed = int.from_bytes(random_data, byteorder="big")
print(f'Using seed: {params.general.pytorch_and_numpy_seed}')

if params.training.device == 'auto':
    params.training.device = 'cuda' if torch.cuda.is_available() else 'cpu'

Using seed: 3815481323


In [3]:
from TransformerMOT.util.misc import NestedTensor

data_generator = DataGenerator(params=params)
training_nested_tensor, labels, unique_measurement_ids, target_coordinates = data_generator.get_batch()

zeros_tensors = torch.zeros_like(training_nested_tensor.tensors)
test_nested_tensor = NestedTensor(zeros_tensors, training_nested_tensor.mask).to("cuda")


In [4]:
print(training_nested_tensor.tensors.shape)
print(target_coordinates.shape)

torch.Size([5, 513, 4])
torch.Size([5, 513, 2])


In [12]:
model1 = BOMT(params).to("cuda")
checkpoint = torch.load(r"results\train2_2phases_1\checkpoints\checkpoint_gradient_step_1615.pth",map_location="cuda")
model1.load_state_dict(checkpoint["model_state_dict"])
res = model1.forward_phase1(training_nested_tensor.to("cuda"))

In [13]:
res[0,:,:]

tensor([[37.9610, 36.8460],
        [38.1306, 36.4190],
        [38.3173, 36.4238],
        ...,
        [-0.2672, -0.7491],
        [ 0.4266,  0.7768],
        [ 0.9106,  1.1746]], device='cuda:0', grad_fn=<SliceBackward0>)

In [10]:
target_coordinates[0,:,:]

tensor([[30.7874, 63.7404],
        [47.4985, 16.8110],
        [32.4590, 65.1297],
        ...,
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000]], device='cuda:0')

In [6]:
model1 = BOMT(params).to("cuda")
checkpoint = torch.load(r"results\train2_new\checkpoints\checkpoint_gradient_step_2005.pth",map_location="cuda")
model1.load_state_dict(checkpoint["model_state_dict"])
res = model1(training_nested_tensor.to("cuda"))
res_1 = model1(test_nested_tensor.to("cuda"))

In [7]:
res[0].positions

tensor([[[49.4108, 50.8714],
         [49.9599, 51.4210],
         [49.7491, 51.1946],
         [49.8876, 51.3311],
         [50.0013, 51.4702],
         [49.6534, 51.1106],
         [49.8619, 51.3162],
         [49.5199, 50.9331],
         [49.2935, 50.7135],
         [50.1319, 51.5960],
         [49.4155, 50.8675],
         [49.7803, 51.2075],
         [49.8832, 51.3492],
         [50.5219, 52.0068],
         [49.9429, 51.4053],
         [50.2242, 51.6975]],

        [[49.1723, 50.6165],
         [50.1208, 51.5944],
         [49.8042, 51.2548],
         [49.3624, 50.7905],
         [49.8500, 51.2913],
         [49.4856, 50.9412],
         [50.6588, 52.1333],
         [50.1576, 51.6006],
         [49.8645, 51.3294],
         [48.8064, 50.2233],
         [49.7372, 51.1929],
         [50.0376, 51.5131],
         [49.7521, 51.2047],
         [49.6538, 51.0704],
         [49.9513, 51.4027],
         [49.0465, 50.4809]],

        [[49.0739, 50.4968],
         [49.3136, 50.7397],
         [

In [8]:
model1.rf_layer.d_embeddings.weight

Parameter containing:
tensor([[-0.2513],
        [-0.1306],
        [ 0.0383],
        [ 0.2029],
        [ 0.0343],
        [ 0.0881],
        [-0.1261],
        [-0.1580],
        [-0.2722],
        [ 0.0630],
        [-0.1347],
        [ 0.1349],
        [ 0.0363],
        [-0.1939],
        [-0.0149],
        [-0.0254],
        [-0.0559],
        [-0.2344],
        [ 0.0210],
        [ 0.2574],
        [ 0.2565],
        [-0.2365],
        [-0.1967],
        [-0.1749],
        [ 0.2456],
        [-0.1519],
        [-0.0932],
        [ 0.2648],
        [-0.2740],
        [-0.1737],
        [ 0.2630],
        [ 0.2208],
        [-0.0836],
        [ 0.0826],
        [-0.0648],
        [-0.2234],
        [-0.2156],
        [ 0.2931],
        [-0.1939],
        [-0.2458],
        [-0.2658],
        [ 0.2779],
        [-0.2080],
        [-0.0807],
        [-0.0223],
        [ 0.1780],
        [ 0.2675],
        [-0.2822],
        [ 0.0521],
        [ 0.2838],
        [-0.3008],
        [

In [9]:
res[-1]

{'intermediate_attention': []}